In [1]:
!pip install tqdm


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install kagglehub


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import torch as nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

#Download the data in the cache

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yudhaislamisulistya/plants-type-datasets")

print("Path to dataset files:", path)

100%|██████████| 937M/937M [00:25<00:00, 38.7MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/yudhaislamisulistya/plants-type-datasets/versions/16


In [5]:
import shutil, os

SRC = path
DST = "/workspace/plants_dataset"

if not os.path.exists(DST):
    shutil.copytree(SRC, DST)

print("Dataset copied to local SSD")

Dataset copied to local SSD


In [6]:
base_path = "/workspace/plants_dataset/split_ttv_dataset_type_of_plants"
os.listdir(base_path)

['Validation_Set_Folder', 'Train_Set_Folder', 'Test_Set_Folder']

In [ ]:
# import os

# path = "/root/.cache/kagglehub/datasets/yudhaislamisulistya/plants-type-datasets/versions/16"
# os.listdir(path)

['split_ttv_dataset_type_of_plants']

In [ ]:
# os.listdir(path)

['split_ttv_dataset_type_of_plants']

In [ ]:
# base_path = "/root/.cache/kagglehub/datasets/yudhaislamisulistya/plants-type-datasets/versions/16/split_ttv_dataset_type_of_plants"
# os.listdir(base_path)

['Test_Set_Folder', 'Train_Set_Folder', 'Validation_Set_Folder']

In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Using device: cuda
GPU name: NVIDIA RTX A4500


#Do the transforms and do the data set

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

vald_transform = transforms.Compose({
    transforms.Resize((224,224)),
    transforms.ToTensor()
})

test_transform = transforms.Compose({
    transforms.Resize((224,224)),
    transforms.ToTensor()
})

In [9]:
Train_dataset = ImageFolder(
    root=os.path.join(base_path, "Train_Set_Folder"),
    transform=train_transform)

In [10]:
Train_dataset

Dataset ImageFolder
    Number of datapoints: 23972
    Root location: /workspace/plants_dataset/split_ttv_dataset_type_of_plants/Train_Set_Folder
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [11]:
val_dataset = ImageFolder(
    root=os.path.join(base_path, "Validation_Set_Folder"),
    transform=vald_transform)

In [12]:
val_dataset

Dataset ImageFolder
    Number of datapoints: 3030
    Root location: /workspace/plants_dataset/split_ttv_dataset_type_of_plants/Validation_Set_Folder
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [13]:
test_dataset = ImageFolder(
    root=os.path.join(base_path, "Test_Set_Folder"),
    transform=test_transform)

In [14]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2998
    Root location: /workspace/plants_dataset/split_ttv_dataset_type_of_plants/Test_Set_Folder
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
           )

# the Dataloader

In [15]:
# train_loader = DataLoader(Train_dataset ,batch_size = 512 , shuffle =True)
# val_loader = DataLoader(val_dataset,batch_size = 512 , shuffle =False)
# test_loader = DataLoader(test_dataset,batch_size = 512 , shuffle =False)

In [16]:
train_loader = DataLoader(
    Train_dataset,
    batch_size=512,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=512,
    shuffle=False,
    num_workers=8,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=512,
    shuffle=False,
    num_workers=8,
    pin_memory=True
)

#bulid the neural network

In [17]:
class Osama_model(nn.Module):
  def __init__(self,num_classes = 30):
    super().__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(3 * 224 * 224, 512)
    self.fc2 = nn.Linear(512, 128)
    self.fc3 = nn.Linear(128, num_classes)

    self.relu = nn.ReLU()

  def forward(self , x):
    x = self.flatten(x)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [18]:
the_Goat = Osama_model()
the_Goat = the_Goat.to(device)
the_Goat

Osama_model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=150528, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=30, bias=True)
  (relu): ReLU()
)

# the lossy ossy

In [19]:
optimizer = optim.Adam(the_Goat.parameters() , lr = 0.0001)
Loss_function = nn.CrossEntropyLoss()

# for loop time

just training

In [39]:
Epochs = 60

for epoch in range(Epochs):
  epoch_loss = 0.0
  correct = 0
  total = 0

  for input, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{Epochs}"):
    input  = input.to(device)
    target = target.to(device)


    optimizer.zero_grad()
    output = the_Goat(input)
    loss = Loss_function(output , target)
    epoch_loss += loss.item()
    loss.backward()
    optimizer.step()


    preds = output.argmax(dim=1)
    correct += (preds == target).sum().item()
    total += target.size(0)


  epoch_loss /= len(train_loader)
  accuracy = correct / total


  print(
        f"Epoch {epoch+1}/{Epochs} | "
        f"Loss: {epoch_loss:.4f} | "
        f"Accuracy: {accuracy*100:.2f}%"
    )

Epoch 1/60:   0%|          | 0/47 [00:16<?, ?it/s]


KeyboardInterrupt: 

Training + val

In [20]:
Epochs = 60

for epoch in range(Epochs):

    # =======================
    # Training
    # =======================
    the_Goat.train()
    epoch_loss = 0.0
    correct = 0
    total = 0

    for input, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{Epochs} [Train]"):
        input  = input.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = the_Goat(input)
        loss = Loss_function(output, target)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        preds = output.argmax(dim=1)
        correct += (preds == target).sum().item()
        total += target.size(0)

    train_loss = epoch_loss / len(train_loader)
    train_acc  = correct / total

    # =======================
    # Validation
    # =======================
    the_Goat.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for input, target in tqdm(val_loader, desc=f"Epoch {epoch+1}/{Epochs} [Val]"):
            input  = input.to(device)
            target = target.to(device)

            output = the_Goat(input)
            loss = Loss_function(output, target)

            val_loss += loss.item()

            preds = output.argmax(dim=1)
            val_correct += (preds == target).sum().item()
            val_total += target.size(0)

    val_loss /= len(val_loader)
    val_acc  = val_correct / val_total

    # =======================
    # Print results
    # =======================
    print(
        f"Epoch {epoch+1}/{Epochs} | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}% || "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%"
    )


Epoch 1/60 [Val]: 100%|██████████| 6/6 [00:07<00:00,  1.17s/it]


Epoch 1/60 | Train Loss: 3.4124 | Train Acc: 6.40% || Val Loss: 3.2224 | Val Acc: 10.26%


Epoch 2/60 [Val]: 100%|██████████| 6/6 [00:04<00:00,  1.24it/s]


Epoch 2/60 | Train Loss: 3.1422 | Train Acc: 12.39% || Val Loss: 3.0742 | Val Acc: 13.33%


Epoch 3/60 [Val]: 100%|██████████| 6/6 [00:05<00:00,  1.17it/s]


...

Epoch 58/60 | Train Loss: 1.7424 | Train Acc: 47.83% || Val Loss: 2.2849 | Val Acc: 30.07%


Epoch 59/60 [Val]: 100%|██████████| 6/6 [00:05<00:00,  1.19it/s]


Epoch 59/60 | Train Loss: 1.7338 | Train Acc: 48.30% || Val Loss: 2.2794 | Val Acc: 30.76%


Epoch 60/60 [Val]: 100%|██████████| 6/6 [00:04<00:00,  1.26it/s]

Epoch 60/60 | Train Loss: 1.7150 | Train Acc: 49.26% || Val Loss: 2.3023 | Val Acc: 29.74%


In [21]:
the_Goat.eval()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = the_Goat(inputs)
        loss = Loss_function(outputs, targets)

        test_loss += loss.item()

        preds = outputs.argmax(dim=1)
        correct += (preds == targets).sum().item()
        total += targets.size(0)

test_loss /= len(test_loader)
test_acc = correct / total

print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc*100:.2f}%")


Test Loss: 2.2779 | Test Accuracy: 31.15%
